# 4.3 Data Wrangling Exercise


## Submission questions

Create a short document (1-2 pages) in your github describing the data wrangling steps that you undertook to clean your capstone project data set. What kind of cleaning steps did you perform? How did you deal with missing values, if any? Were there outliers, and how did you decide to handle them? This document will eventually become part of your milestone report.

## Approach

1. Data exploration
2. Data cleaning
3. Data transformation


### 1. and 2. Data exploration and cleaning

Data exploration involves loading the data, checking the data types of different variables, importing packages as necessary. 

In [2]:
#import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

First few rows of the data are displayed below.

In [3]:
#read the data file
input_file_name = 'diabetes.csv'
rawdata_df = pd.read_csv(input_file_name, na_values = "?", engine='python')
rawdata_df.head(10)

,encounter_id,patient_nbr,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),NaN,6,25,1,1,...,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),NaN,1,1,7,3,...,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),NaN,1,1,7,2,...,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),NaN,1,1,7,2,...,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),NaN,1,1,7,1,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
5,35754,82637451,Caucasian,Male,[50-60),NaN,2,1,2,3,...,No,Steady,No,No,No,No,No,No,Yes,>30
6,55842,84259809,Caucasian,Male,[60-70),NaN,3,1,2,4,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
7,63768,114882984,Caucasian,Male,[70-80),NaN,1,1,7,5,...,No,No,No,No,No,No,No,No,Yes,>30
8,12522,48330783,Caucasian,Female,[80-90),NaN,2,1,4,13,...,No,Steady,No,No,No,No,No,Ch,Yes,NO
9,15738,63555939,Caucasian,Female,[90-100),NaN,3,3,4,12,...,No,Steady,No,No,No,No,No,Ch,Yes,NO


Features with missing values ( > 96%) are eliminated.

In [4]:
#make a copy of the dataframe
cleandata_df = rawdata_df.copy()

#checking for the number of missing values per feature (if > 0)
for x in cleandata_df.columns:
    is_object = cleandata_df[x].dtype
    if (is_object == object and 
        cleandata_df[x][cleandata_df[x] == '?'].count() > 0):
        print(x,': % missing values - ', 
              cleandata_df[x][cleandata_df[x] == '?'].count()*100/cleandata_df[x].count())

In [5]:
#drop the feature 'weight'
cleandata_df.drop(['weight'], inplace = True, axis = 1)

In [6]:
#check the shape again 
cleandata_df.shape

(101766, 49)

Unknown/missing values are replaced with NaN.

In [7]:
#replace missing values with NaN
cleandata_df.replace('Unknown/Invalid', np.NaN, inplace = True)

In [8]:
#checking if '?'s are replaced with NaN
for z in cleandata_df.columns:
    is_object1 = cleandata_df[z].dtype
    if is_object1 == object and cleandata_df[z][cleandata_df[z] == '?'].count() > 0:
        print(z, ': % missing values - ', 
              cleandata_df[z][cleandata_df[z] == '?'].count()*100/cleandata_df[z].count())

### 3. Data Transformation

Columns where replacement/recoding of values were necessary were identified and replaced with new codes.
For example, the column "Readmitted" with values "No", ">30", and "<30" were replaced with "0" for "No" and ">30" and "1" for "<30".

In [9]:
#'Age' feature
age_values = {"age":{"[0-10)": 1, "[10-20)": 2, "[20-30)": 3, 
                    "[30-40)": 4, "[40-50)": 5, "[50-60)": 6,
                    "[60-70)": 7, "[70-80)": 8, "[80-90)": 9,
                    "[90-100)": 10}}
cleandata_df.replace(age_values, inplace = True)

In [10]:
#checking for number of readmissions within 30 days
cleandata_df['readmitted'].value_counts()

NO     54864
>30    35545
<30    11357
Name: readmitted, dtype: int64

In [11]:
#converting readmitted < 30 days value to 1 and rest to 0
readmitted_values = {"readmitted": {"NO": 0, ">30": 0, "<30": 1}}
cleandata_df.replace(readmitted_values, inplace = True)

In [12]:
#checking for new values
cleandata_df['readmitted'].value_counts()

0    90409
1    11357
Name: readmitted, dtype: int64

In [13]:
#writing the dataframe into csv for further analysis
output_file_name = "clean_data_diabetes.csv"
cleandata_df.to_csv(output_file_name, sep = '\t')